In [1]:
from fz_text_seg import tokenization, lexical_score

In [12]:
text_tokens, unique_tokens, paragraphs = tokenization('corpus.txt', 13)
len(text_tokens)

18

In [14]:
lex_score = lexical_score(text_tokens, unique_tokens, 5)
lex_score

[0. 0. 0. 0. 0.]
[0.         0.03333333 0.03333333 0.03333333 0.03333333]
[0.03333333 0.06814886 0.06814886 0.06814886 0.06814886]
[0.03333333 0.06814886 0.06814886 0.06814886 0.06814886]
[0.06814886 0.06814886 0.09701638 0.09701638 0.06368304]
[0.06814886 0.06814886 0.09701638 0.09701638 0.06368304 0.02886751]
[0.06814886 0.06814886 0.09701638 0.09701638 0.13034971 0.09553418
 0.09553418]
[0.06814886 0.06814886 0.09701638 0.09701638 0.13034971 0.12639485
 0.12639485 0.09752734]
[0.06814886 0.06814886 0.09701638 0.09701638 0.13034971 0.12639485
 0.13834714 0.10947962 0.10947962]
[0.04166667 0.08518608 0.08518608 0.12127047 0.07960381 0.11941773
 0.15799356 0.13684953 0.19275123 0.10941789]
[0.05555556 0.11358144 0.11358144 0.10613841 0.04811252 0.15922363
 0.16254556 0.18246604 0.14589053 0.1590058  0.13908532]
[0.         0.08333333 0.17037216 0.08703883 0.07216878 0.07216878
 0.16666667 0.24381834 0.10703239 0.14168411 0.20862798 0.12909944]
[0.         0.16666667 0.17407766 0.      

[0.0,
 0.16666666666666666,
 0.17407765595569785,
 0.0,
 0.14433756729740646,
 0.0,
 0.3333333333333333,
 0.1543033499620919,
 0.05976143046671968,
 0.22360679774997896,
 0.19364916731037085,
 0.06454972243679027,
 0.05976143046671968]